[View in Colaboratory](https://colab.research.google.com/github/zezzer/machine-learning-portfolio/blob/master/MNIST_Keras_CNN.ipynb)

In [0]:
import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.datasets import mnist
import numpy as np

%matplotlib inline

In [0]:
# PARAMETERS
img_width = 28
img_height = 28
batch_size = 100
epochs = 15
seed = 1
ratio_train = 0.8

In [0]:
np.random.seed(seed) # for reproducibility

In [14]:
# Read in image and label data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


# Partition into validation and training sets
val_images = train_images[int(ratio_train*len(train_images)):]
val_labels = train_labels[int(ratio_train*len(train_labels)):]
train_images = train_images[:int(ratio_train*len(train_images))]
train_labels = train_labels[:int(ratio_train*len(train_labels))]

# Reshape
train_images = train_images.reshape(train_images.shape[0], img_width, img_height, 1).astype('float32')
val_images = val_images.reshape(val_images.shape[0], img_width, img_height, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], img_width, img_height, 1).astype('float32')

# Normalize the data
train_images = train_images/255.0
val_images = val_images/255.0
test_images = test_images/255.0

# One hot encode the labels for multi-class classification, for expressive data representation
train_labels = keras.utils.to_categorical(train_labels)
val_labels = keras.utils.to_categorical(val_labels)
test_labels = keras.utils.to_categorical(test_labels)

print(train_images.shape)
print(val_images.shape)
print(test_images.shape)

(48000, 28, 28, 1)
(12000, 28, 28, 1)
(10000, 28, 28, 1)


In [0]:
model = Sequential()

# General notes (applying miscellaneous tips garnered from reading Stanford CS231n notes):
# Keep convolutional filter sizes small & stack convolutional layers for more expressive features.
# Padding is added to the Conv2D layers for clean design, so only MaxPooling is in charge of downsampling.
# Relu activation is used for efficient computations.
# Dropout is used to reduce interdependence between neurons.

model.add(Conv2D(32, kernel_size=(3,3), padding='same',
                activation='relu',
                input_shape=(img_width, img_height, 1)))
model.add(Conv2D(32, kernel_size=(3,3), padding='same',
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3,3), padding='same',
                activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), padding='same',
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3,3), padding='same',
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

# Fully connected layers that use extracted features for classification
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
history = model.fit(train_images, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(val_images, val_labels), 
          shuffle = True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/15
48000/48000 [==============================] - 252s 5ms/step - loss: 0.4415 - acc: 0.8540 - val_loss: 0.0808 - val_acc: 0.9750
Epoch 2/15
48000/48000 [==============================] - 251s 5ms/step - loss: 0.1210 - acc: 0.9643 - val_loss: 0.0425 - val_acc: 0.9882
Epoch 3/15
48000/48000 [==============================] - 251s 5ms/step - loss: 0.0903 - acc: 0.9746 - val_loss: 0.0341 - val_acc: 0.9908
Epoch 4/15
48000/48000 [==============================] - 250s 5ms/step - loss: 0.0737 - acc: 0.9789 - val_loss: 0.0348 - val_acc: 0.9901
Epoch 5/15
48000/48000 [==============================] - 249s 5ms/step - loss: 0.0650 - acc: 0.9810 - val_loss: 0.0350 - val_acc: 0.9896
Epoch 6/15
48000/48000 [==============================] - 248s 5ms/step - loss: 0.0554 - acc: 0.9836 - val_loss: 0.0368 - val_acc: 0.9895
Epoch 7/15
48000/48000 [==============================] - 249s 5ms/step - loss: 0.0515 - acc: 0.9852 - val_loss: 0.0271 - 

In [0]:
score = model.evaluate(test_images, test_labels, verbose=0)

In [22]:
print(score)

[0.01892832144598851, 0.9943]
